In [16]:
def metric_dimension(G):
    n = G.num_verts()
    D = G.distance_matrix()
    p = MixedIntegerLinearProgram(maximization = False)
    x = p.new_variable(binary = True)
    for u, v in Combinations(range(n), 2):
        p.add_constraint(sum(x[w] for w in range(n)
                             if D[u, w] != D[v, w]) >= 1)
    p.set_objective(sum(x[w] for w in range(n)))
    return p.solve()

In [10]:
G = graphs.CompleteGraph(3)
metric_dimension(G)

2.0

In [11]:
G = graphs.CompleteGraph(12)
metric_dimension(G)

11.0

In [15]:
H = graphs.PetersenGraph()
metric_dimension(H)

3.0

In [3]:
I = graphs.CompleteBipartiteGraph(3,2)
metric_dimension(I)

3.0

In [5]:
J = graphs.CubeGraph(3)
metric_dimension(J)

3.0

In [7]:
K = graphs.CycleGraph(5)
metric_dimension(K)

2.0

In [10]:
L = graphs.PathGraph(10)
metric_dimension(L)

1.0